In [ ]:
# default_exp projects

# Projects

Tight.ai projects are essentially your ML models as production-ready cloud services. 

Projects are serverless machine learning applications as REST API endpoints made for scalability, high availability, and improvement simplicity.

Serverless means you pay for what you use, when you use it thus drastically reducing the cost of every serverless app you have in production.

In [ ]:
#hide
# !pip install python-slugify

In [ ]:
#export
import time
import urllib.parse
import webbrowser
from pathlib import Path
from slugify import slugify
from tightai import exceptions
from tightai.exceptions import ArgumentsRequired
from tightai.lookup import Lookup
from tightai.conf import CLI_ENDPOINT
from tightai.upload import Uploader
from tightai.utils import sort_by_key, print_dict

In [ ]:
#hide
test = False
if test:
    CLI_ENDPOINT = "http://cli.desalsa.io:8000"

In [ ]:
#export
base_tightignore = ['# IPython\n', 'share/python-wheels/\n', '\n', '.hypothesis/\n', '# PyInstaller\n', '.scrapy\n', '.ipynb_checkpoints\n', '# SageMath parsed files\n', '.gitconfig', '.tox/\n', '*~\n', '*.bak\n', '.python-version\n', '# .python-version\n', '# Spyder project settings\n', '__pycache__/\n', '*.swp\n', '*.cover\n', 'instance/\n', '# pytest\n', 'celerybeat.pid\n', 'tools/fastai\n', '*.py,cover\n', '.env\n', '*.py[cod]\n', '# dotenv\n', 'dmypy.json\n', '# Flask stuff:\n', '.coverage.*\n', 'db.sqlite3-journal\n', 'Thumbs.db\n', '*.code-workspace\n', '# virtualenv\n', '.spyproject\n', '# pytype static type analyzer\n', '#   According to pypa/pipenv#598, it is recommended to include Pipfile.lock in version control.\n', 'coverage.xml\n', '#   install all needed dependencies.\n', '.vscode/\n', '#   intended to run in multiple environments; otherwise, check them in:\n', '*.egg\n', '# mypy\n', 'celerybeat-schedule\n', '.mypy_cache/\n', '# Environments\n', 'htmlcov/\n', '*.log\n', '# Scrapy stuff:\n', '*.whl\n', 'tags\n', '.cache\n', 'env.bak/\n', '# pipenv\n', 'venv.bak/\n', 'cython_debug/\n', 'MANIFEST\n', '.DS_Store?\n', '.pybuilder/\n', '.pyre/\n', '.webassets-cache\n', 'bin/\n', '.ropeproject\n', 'ipython_config.py\n', '# Sphinx documentation\n', '# link checker\n', '#  Usually these files are written by a python script from a template\n', '~*\n', 'develop-eggs/\n', '# Django stuff:\n', '.spyderproject\n', '# symlinks to fastai\n', '# Cython debug symbols\n', '# mkdocs documentation\n', '# Byte-compiled / optimized / DLL files\n', 'cover/\n', '*.mo\n', 'docs_src/fastai\n', '# PyBuilder\n', '.nox/\n', '# Rope project settings\n', '.idea\n', 'lib/\n', '.pytest_cache\n', "#   having no cross-platform support, pipenv may install dependencies that don't work, or not\n", 'env/\n', 'pip-delete-this-directory.txt\n', 'docs/\n', 'tmp*\n', '*.pot\n', 'wheels/\n', 'ENV/\n', 'pip-log.txt\n', 'venv/\n', '/site\n', '# C extensions\n', 'var/\n', '.Trashes\n', 'downloads/\n', 'checklink/cookies.txt\n', '#Pipfile.lock\n', '# osx generated files\n', '.coverage\n', 'eggs/\n', '.venv\n', 'parts/\n', '*.spec\n', '.pytype/\n', '_tmp*\n', '.installed.cfg\n', '# tools/trust-doc-nbs\n', '*.so\n', '*.md\n', '# Installer logs\n', 'lib64/\n', '.Python\n', '*$py.class\n', 'docs_src/.last_checked\n', '.pytest_cache/\n', 'local_settings.py\n', '#   However, in case of collaboration, if having platform-specific dependencies or dependencies\n', '.dmypy.json\n', '# .gitconfig is now autogenerated\n', '# Celery stuff\n', '#  before PyInstaller builds the exe, so as to inject date/other infos into it.\n', '.gitattributes\n', 'docs/_build/\n', '# PEP 582; used by e.g. github.com/David-OConnor/pyflow\n', 'build/\n', '# Distribution / packaging\n', '# Pyre type checker\n', '# Translations\n', '*.manifest\n', '.last_checked\n', '.git/\n', '.DS_Store\n', 'sdist/\n', 'dist/\n', 'target/\n', '__pypackages__/\n', '.gitconfig\n', '.github\n', '.eggs/\n', '*.sage.py\n', 'db.sqlite3\n', 'nosetests.xml\n', 'ehthumbs.db\n', '# pyenv\n', '*.egg-info/\n', '# celery beat schedule file\n', '#   For a library or package, you might want to ignore these files since the code is\n', '.gitignore\n', 'profile_default/\n', '# Unit test / coverage reports\n', '# Jupyter Notebook\n', '.vscode\n']

In [ ]:
#export
class BaseObject():
    _user_keys = ['user', 'owner', 'created_by', 'added_by']
    display_options = []
    def __init__(self, *args, **kwargs):
        super().__init__()
        for k,v in kwargs.items():
            setattr(self, k, v)
    
    def __repr__(self):
        return self.__str__()
    
    def __str__(self):
        return f"<Model: {self.name}>"
    
    def details(self, verbose=1):
        item_dict = self.__dict__
        d = {}
        for k, v in item_dict.items():
            if k in self.display_options:
                d[k] = v
        if verbose == 1:
            print_dict(item_dict)
        return d

In [ ]:
#export
class UserObject(BaseObject):
    def __str__(self):
        return f"<User: {self.username}>"

In [ ]:
#export
class VersionManager(Lookup, BaseObject):
    api = CLI_ENDPOINT
    version = None
    project_id = None
    
    def __str__(self):
        return f"<VersionManager>"
    
    def __repr__(self):
        return self.__str__()
    
    def create(self, project_id=None, raw=False, **kwargs):
        if project_id == None:
            raise ArgumentsRequired("project_id")
        r = self.http_post(f"/projects/{project_id}/versions/", data=kwargs)
        if raw:
            return r
        self.handle_invalid_lookup(r, expected_status_code=201)
        return Version(**r.json())
    
    def get(self, version=None, project_id=None, raw=False):
        if project_id == None or version == None:
            raise ArgumentsRequired("version", "project_id")
        r = self.http_get(f"/projects/{project_id}/versions/{version}")
        if raw:
            return r
        self.handle_invalid_lookup(r, expected_status_code=200)
        return Version(**r.json())
    
    def latest(self, project_id=None, none_on_404=False, deployed=False):
        if project_id == None:
            raise ArgumentsRequired("project_id")
        deployed_str = f"{deployed}".lower()
        r = self.http_get(f"/projects/{project_id}/latest/?deployed={deployed_str}")
        if deployed == True:
            none_on_404 = True
        if r.status_code == 404 and none_on_404:
            return None
        self.handle_invalid_lookup(r, expected_status_code=200)
        return Version(**r.json())
    
    def all(self, project_id=None, **kwargs):
        if project_id == None:
            raise ArgumentsRequired("project_id")
        endpoint = f"/projects/{project_id}/versions/"
        if len(kwargs.keys()):
            query_string = urllib.parse.urlencode(kwargs)
            endpoint = f"{endpoint}?{query_string}"            
        r = self.http_get(endpoint)
        self.handle_invalid_lookup(r, expected_status_code=200)
        return [Version(**x) for x in r.json()['results']]

    def get_or_create(self, **kwargs):
        created = False
        version = None
        try:
            existing_r = self.get(**kwargs)
        except:
            existing_r = None
        if existing_r != None:
            return existing_r, created
        new_v = self.create(**kwargs)
        created = True
        return new_v, created
    

class Version(Lookup, BaseObject):
    version = None
    project_id = None
    trigger_options = ['status', 'build', 'build_and_deploy', 'deploy', 'destroy']
    display_options = ['project_id', 'version', 'description', 'url', 'deployed', 
                       'deployed_timestamp', 'destroyed', 'destroyed_timestamp', 'updated', 'timestamp']
    
    def __str__(self):
        return f"<Version {self.version} ({self.project_id})>"
    
    objects = VersionManager()
    
    def build(self):
        return self.update_trigger(option='build')
    
    def delete(self):
        confirmation_code = f"{self.project_id}-v{self.version}"
        response = input(f"Deleting is cannot be reversed. Please confirm by typing:\n\n{confirmation_code}\n\n")
        if f"{response}".strip() != confirmation_code:
            print(f"\nDelete failed. {response} did not match {confirmation_code} exactly.")
            return 
        r = self.http_delete(f"/projects/{self.project_id}/versions/{self.version}")
        self.handle_invalid_lookup(r, expected_status_code=204)
        setattr(self, 'deleted', True)
        return r
    
    def summary(self, verbose=0):
        data = {
            "version": self.version,
            "project_id": self.project_id,
            "url": self.url,
            "deployed": self.deployed,
            "updated": self.updated
        }
        if verbose == 1:
            print_dict(data)
        return data
    
    def deploy(self):
        raise Exception("Use `.push(local_path)` to deploy instead.")
    
    def destroy(self):
        return self.update_trigger(option='destroy')
    
    def upload(self, path, verbose=True):
        src_path = Path(path)
        tightai_ignore = src_path / ".tightignore"
        if not tightai_ignore.exists():
            print("No .tightignore found. Creating default.")
            with open(str(tightai_ignore), "w") as f:
                f.writelines(base_tightignore)
        uploader = Uploader(path=path, project_id=self.project_id, version=self.version)
        uploaded_count = uploader.upload(verbose=verbose)
        return uploaded_count
    
    def status(self, wait=True, display=True):
        endpoint = f"/projects/{self.project_id}/versions/{self.version}/status/"
        r = self.http_post(endpoint, data={})
        if r.status_code != 200:
            raise Exception("Could not find status at this time.")
        r_data = r.json()
        for k,v in r_data.items():
            setattr(self, k, v)
        verbose = 1
        if display == False:
            verbose = 0
        return self.details(verbose=verbose)
    
    def refresh(self):
        version = self.objects.get(version=self.version, project_id=self.project_id)
        for k,v in version.__dict__.items():
            setattr(self, k, v)
        return version.details()
    
    def push(self, path, check_status=True):
        project_id = self.project_id
        version = self.version
        uploaded_count = self.upload(path, verbose=True)
        if uploaded_count == 0:
            print(f"Uploaded 0 files. Continuing")
        print("Deploying...")
        deploy_r = self.update_trigger(option='build_and_deploy')
        if deploy_r.status_code != 200:
            raise Exception("Project not deployed. Please try again.")
        if not check_status:
            print(f"{project_id} {version} pushed. Use `.status()` to check deployment status.")
            return None
        details = self.status(wait=True)
        url = details.get('url')
        if not url:
            print("URL is not ready yet. Use `.status()` to check deployment status.")
            return None
        setattr(self, 'url', url)
        return url
    
    def update_trigger(self, option='status'):
        if option not in self.trigger_options:
            print(f"{option} is invalid. Please try again.")
            option = 'status'
        data = {
            'trigger_active': True,
            'trigger': f"{option}",
        }
        r = self.http_put(f"/projects/{self.project_id}/versions/{self.version}", data=data)
        self.handle_invalid_lookup(r, expected_status_code=200)
        self.update_from_response(r)
        return r
    
    def update_from_response(self, response, display=False):
        for k, v in response.json().items():
            if display:
                print(f"{k}")
            current_v = getattr(self, k)
            if v != None and current_v != v:
                setattr(self, k, v)
        return
    
    def open(self):
        details = self.details()
        url = details.get("url")
        if url == None or details['deployed'] == False:
            raise Exception("This version has not been deployed yet. Push a deployment or refresh version status.")
        return webbrowser.open(url)
        

In [ ]:
#export
class ProjectManager(Lookup, BaseObject):
    api = CLI_ENDPOINT
    version = None
    
    def __str__(self):
        return f"<ProjectManager>"
    
    def __repr__(self):
        return self.__str__()
        
    def create(self, **kwargs):
        r = self.http_post(f"/projects/", data=kwargs)
        self.handle_invalid_lookup(r, expected_status_code=201)
        return Project(**r.json())
    
    def get(self, project_id=None):
        if project_id == None:
            raise ArgumentsRequired("project_id")
        r = self.http_get(f"/projects/{project_id}/")
        self.handle_invalid_lookup(r, expected_status_code=200)
        return Project(**r.json())
    
    def all(self, **kwargs):
        endpoint = f"/projects/"
        if len(kwargs.keys()):
            query_string = urllib.parse.urlencode(kwargs)
            endpoint = f"{endpoint}?{query_string}"            
        r = self.http_get(endpoint)
        self.handle_invalid_lookup(r, expected_status_code=200)
        return [Project(**x) for x in r.json()['results']]

    def get_or_create(self, **kwargs):
        created = False
        existing_r = self.get()
        if existing_r.status_code == 200:
            return existing_r, created
        new_r = self.create(**kwargs)
        self.handle_invalid_lookup(new_r, expected_status_code=201)
        created = True
        return new_r, created
    

class Project(Lookup, BaseObject):
    project_id = None
    display_options = ['name', 'id', 'description', 
                       'updated', 'timestamp', 'versions_metadata', 'versions_urls']
    
    def __str__(self):
        return f"<Project {self.id}>"
    
    def summary(self, verbose=0):
        data = {
            "name": self.name,
            "project_id": self.project_id,
            "versions_summary": {
                "deployed": self.versions_metadata.get('deployed_count'),
                "urls": self.versions_urls,
            },
            "updated": self.updated
        }
        if verbose == 1:
            print_dict(data)
        return data
    
    @property
    def project_id(self):
        return self.id
    
    objects = ProjectManager()
    
    def create_version(self, version=None):
        if version == None:
            raise ArgumentsRequired("version")
        version, _ = Version.objects.get_or_create(project_id=self.project_id, version=version)
        return version
    
    def get_version(self, version=None):
        if version == None:
            raise ArgumentsRequired("version")
        version, _ = Version.objects.get_or_create(project_id=self.project_id, version=version)
        return version
    
    def delete(self):
        print("You can only delete projects in the tight.ai console right now.")
    
    def get_versions(self):
        versions = Version.objects.all(project_id=self.project_id)
        return versions
    
    def latest(self, deployed=False):
        return Version.objects.latest(project_id=self.project_id, deployed=deployed)

In [ ]:
proj = Project.objects.get(project_id='news-categories')
proj.project_id

'news-categories'

In [ ]:
proj.summary()

{'name': 'News Categories',
 'project_id': 'news-categories',
 'versions_summary': {'deployed': 1,
  'urls': ['http://news-categories-v5.apps.tight.ml']},
 'updated': '2020-05-14T04:37:04.895133Z'}

In [ ]:
proj.details(verbose=1)

name: News Categories
id: news-categories
owner:
username: jmitchel3
first_name: 
last_name: 
date_joined: 2020-05-11T20:35:48.716644Z
description: 
versions_urls:
	- http://news-categories-v5.apps.tight.ml
versions_metadata:
deployed_count: 1
destroyed_count: 0
other: 6
updated: 2020-05-14T04:37:04.895133Z
timestamp: 2020-05-14T04:35:18.429311Z


{'name': 'News Categories',
 'id': 'news-categories',
 'description': '',
 'versions_urls': ['http://news-categories-v5.apps.tight.ml'],
 'versions_metadata': {'deployed_count': 1, 'destroyed_count': 0, 'other': 6},
 'updated': '2020-05-14T04:37:04.895133Z',
 'timestamp': '2020-05-14T04:35:18.429311Z'}

In [ ]:
proj.get_versions()

In [ ]:
recent_version = proj.latest()
if recent_version != None:
    recent_version.summary(verbose=1)
    print("\n\n")
    recent_version.details()

In [ ]:
recent_version.open()

In [ ]:
version_c2, created = Version.objects.get_or_create(version=92023, project_id="news-categories")
version_c2
print(version_c2, created)
time.sleep(4)
version_c2.delete()
print('deleted')

In [ ]:
try:
    version_c = Version.objects.get(version=92023, project_id="news-categories")
except Exception as e:
    print("Expetected Error:", e.message)
    assert e.status == 404

In [ ]:
try:
    version_c = Version.objects.create(version=99023, project_id="news-categories")
except Exception as e:
    print("Expetected Error:", e.message)
    assert e.status == 400

In [ ]:
version = Version.objects.get(version=99023, project_id="news-categories")
version.details()

In [ ]:
version = Version.objects.get(version=6, project_id="news-categories")
# version.refresh()

In [ ]:
deets = version.details()
import json
json.dumps(deets)

project_id: news-categories
version: 6
description: 
url: None
status_msg: Unknown status. Please try again.
build_started: True
build_start_timestamp: 2020-05-20T21:42:50.716189Z
build_complete: True
build_complete_timestamp: 2020-05-20T21:45:17.077165Z
trigger_active: False
trigger: build_and_deploy
deployed: False
deployed_timestamp: None
destroyed: False
destroyed_timestamp: 2020-05-20T21:42:19.091448Z
updated: 2020-05-20T21:45:17.077583Z
timestamp: 2020-05-19T03:49:04.389957Z


'{"project_id": "news-categories", "version": 6, "description": "", "url": null, "deployed": false, "deployed_timestamp": null, "destroyed": false, "destroyed_timestamp": "2020-05-20T21:42:19.091448Z", "updated": "2020-05-20T21:45:17.077583Z", "timestamp": "2020-05-19T03:49:04.389957Z"}'

In [ ]:
version.push("/Users/jmitch/tight/my-tight-apps/local_dev/")

Uploading...


RequestError: '\n<!doctype html>\n<html lang="en">\n<head>\n  <title>Bad Request (400)</title>\n</head>\n<body>\n  <h1>Bad Request (400)</h1><p></p>\n</body>\n</html>\n (status 400)'

In [ ]:
version.details()

In [ ]:
version.status()

In [ ]:
# version.destroy()

In [ ]:
version.status()

In [ ]:
versions = Version.objects.all(project_id="news-categories", size='10', deployed=True)
versions

In [ ]:
# import requests
# r = requests.post('https://www.tight.ai/api/webhooks/projects/versions/status/', data={})

In [ ]:
d = "{\"app_name\": \"news-categories-v5\", \"address\": {\"url\": \"http://news-categories-v5.apps.svc.cluster.local\"}, \"conditions\": [{\"lastTransitionTime\": \"2020-05-19T03:48:44Z\", \"status\": \"True\", \"type\": \"ConfigurationsReady\"}, {\"lastTransitionTime\": \"2020-05-19T03:48:47Z\", \"status\": \"True\", \"type\": \"Ready\"}, {\"lastTransitionTime\": \"2020-05-19T03:48:47Z\", \"status\": \"True\", \"type\": \"RoutesReady\"}], \"latestCreatedRevisionName\": \"news-categories-v5-rk6bb\", \"latestReadyRevisionName\": \"news-categories-v5-rk6bb\", \"observedGeneration\": 1, \"traffic\": [{\"latestRevision\": true, \"percent\": 100, \"revisionName\": \"news-categories-v5-rk6bb\"}], \"url\": \"http://news-categories-v5.apps.tight.ml\"}"

In [ ]:
import json

In [ ]:
b = json.loads(d)

In [ ]:
b

In [ ]:
import requests
endpoint = "http://127.0.0.1:8123/status"
data = {
    'project_id': "news-categories",
    "version": "6"
}
r = requests.post(endpoint, json=data)
data = r.json()
data

In [ ]:
import requests
endpoint = "http://127.0.0.1:8123/live/status/news-categories/v5"
r = requests.get(endpoint)
data = r.json()
data

In [ ]:
traffic_data = data['status']['traffic']
traffic_data = sorted(traffic_data, key = lambda x: -x['percent'])
traffic_data

In [ ]:
ready = False
for i in data['status']['conditions']:
    _cond_ready = f"{i['status']}".lower() == "true"
    ready = _cond_ready
ready

In [ ]:
sorted(traffic_data, key = lambda x: -x['percent'])